## Tall Format Trait Table
### Columns
* Sitename
* Range
* Column
* Date
* Cultivar
* Trait
* Trait Value

### Notes and ToDos for this table
* Add trait id - where is this located?
* Can wait for feedback on the best way to incorporate TO Mappings
* Remove index - use date column as index?
* Add GDD?

In [1]:
import datetime
import numpy as np
import pandas as pd
import sqlalchemy
import sqlite3

In [4]:
# %pwd
# %cd '/Users/ejcain/UA-AG/phenotypes/terraref-datasets'

/Users/ejcain/UA-AG/phenotypes/terraref-datasets


In [12]:
df_0 = pd.read_csv('data/raw/mac_season_4.csv', low_memory=False)
df_0.shape

(372363, 39)

In [13]:
# df_0.columns

Index(['Unnamed: 0', 'checked', 'result_type', 'id', 'citation_id', 'site_id',
       'treatment_id', 'sitename', 'city', 'lat', 'lon', 'scientificname',
       'commonname', 'genus', 'species_id', 'cultivar_id', 'author',
       'citation_year', 'treatment', 'date', 'time', 'raw_date', 'month',
       'year', 'dateloc', 'trait', 'trait_description', 'mean', 'units', 'n',
       'statname', 'stat', 'notes', 'access_level', 'cultivar', 'entity',
       'method_name', 'view_url', 'edit_url'],
      dtype='object')

In [22]:
cols_to_drop = ['Unnamed: 0', 'checked', 'result_type', 'id', 'citation_id', 'site_id', 'treatment_id', 'city', 
                'scientificname', 'commonname', 'genus', 'species_id', 'cultivar_id', 'author', 'citation_year',
                'treatment', 'time', 'raw_date', 'month', 'year', 'dateloc', 'trait_description', 'units', 'n', 'statname',
                'stat', 'notes', 'access_level', 'entity', 'method_name', 'view_url', 'edit_url']

In [23]:
df_1 = df_0.drop(labels=cols_to_drop, axis=1)
# df_1.head()

In [ ]:
# df_1.shape

In [ ]:
# print(df_1.date.min())
# print(df_1.date.max())

In [ ]:
# Do all dates in the df contain the string (America/Phoenix)?

count = 0
for d in df_1.date.values:
    if 'America/Phoenix' in d:
        count += 1

count

In [ ]:
# Do all the dates start with 2017?

count = 0
for d in df_1.date.values:
    if d.startswith('2017'):
        count += 1
        
count

### I. Change date values to iso date format
* date(s) as index?

In [24]:
new_dates = []

for d in df_1.date.values:
    
    if 'Phoenix' in d:
        new_name = d[:-18]
        new_dates.append(new_name)
    
    else:
        new_name = d
        new_dates.append(new_name)
        
print(len(new_dates))

372363


In [25]:
iso_format_dates = pd.to_datetime(new_dates)

In [26]:
df_1['date_1'] = iso_format_dates
df_1.head()

,sitename,lat,lon,date,trait,mean,cultivar,date_1
0,MAC Field Scanner Season 4 Range 11 Column 5,33.074907,-111.974982,2017 Jun 14 (America/Phoenix),leaf_desiccation_present,0.0,PI181083,2017-06-14
1,MAC Field Scanner Season 4 Range 11 Column 6,33.074907,-111.974966,2017 Jun 14 (America/Phoenix),leaf_desiccation_present,0.0,PI564163,2017-06-14
2,MAC Field Scanner Season 4 Range 11 Column 9,33.074907,-111.974917,2017 Jun 14 (America/Phoenix),leaf_desiccation_present,0.0,PI52606,2017-06-14
3,MAC Field Scanner Season 4 Range 11 Column 11,33.074907,-111.974884,2017 Jun 14 (America/Phoenix),leaf_desiccation_present,0.0,PI533792,2017-06-14
4,MAC Field Scanner Season 4 Range 11 Column 14,33.074907,-111.974835,2017 Jun 14 (America/Phoenix),leaf_desiccation_present,0.0,PI535794,2017-06-14


### II. Change sitenames to strip E and W subplot designations
* Check which traits are associated with subplots

In [ ]:
e_w_plots = df_1.loc[(df_1.sitename.str.endswith(' E')) | (df_1.sitename.str.endswith(' W'))]
e_w_plots.trait.unique()

In [ ]:
e_w_plot_traits = e_w_plots.trait.unique()

In [27]:
sitename_values = df_1.sitename.values
no_e_w_names = []

for name in sitename_values:
    
    if name.endswith(' W') | name.endswith(' E'):
        name = name[:-2]
        no_e_w_names.append(name)
        
    else:
        no_e_w_names.append(name)

print(len(no_e_w_names))

372363


In [28]:
df_1['sitename_1'] = no_e_w_names
# df_1.sample(n=7)

In [ ]:
# df_1.loc[df_1.sitename.str.endswith(' E')].iloc[0]

#### III. Extract Range and Column Values

In [29]:
df_1['range'] = df_1['sitename'].str.extract("Range (\d+)").astype(int)
df_1['column'] = df_1['sitename'].str.extract("Column (\d+)").astype(int)

# df_1.sample(n=7)

#### Drop duplicated Columns, Reorder, Rename, & Sort by Date

In [30]:
df_2 = df_1.drop(labels=['sitename', 'date'], axis=1)
df_2.shape

(372363, 9)

In [ ]:
df_2.columns

In [31]:
col_reorder = ['date_1', 'sitename_1', 'range', 'column', 'lat', 'lon', 'cultivar', 'trait', 'mean']

In [32]:
df_3 = pd.DataFrame(data=df_2, columns=col_reorder, index=df_2.index)
# df_3.head()

In [33]:
df_4 = df_3.rename({'date_1': 'date', 'sitename_1': 'sitename', 'mean': 'value'}, axis=1)
df_4.tail()

,date,sitename,range,column,lat,lon,cultivar,trait,value
372358,2017-06-06,MAC Field Scanner Season 4 Range 44 Column 9,44,9,33.076093,-111.974917,PI329286,leaf_angle_beta,2.052416
372359,2017-06-06,MAC Field Scanner Season 4 Range 44 Column 9,44,9,33.076093,-111.974917,PI329286,leaf_angle_chi,2.140986
372360,2017-06-06,MAC Field Scanner Season 4 Range 44 Column 13,44,13,33.076093,-111.974852,PI329843,leaf_angle_mean,0.401801
372361,2017-06-06,MAC Field Scanner Season 4 Range 44 Column 13,44,13,33.076093,-111.974852,PI329843,leaf_angle_alpha,3.217374
372362,2017-06-06,MAC Field Scanner Season 4 Range 44 Column 13,44,13,33.076093,-111.974852,PI329843,leaf_angle_beta,1.980157


In [34]:
df_5 = df_4.sort_values(by=['date'], ascending=True)
print(df_5.shape)
print(df_4.shape)
# df_5.head()

(372363, 9)
(372363, 9)


### III. Drop traits not needed at this time
* dropping `canopy_height` because of the E W subplots - dealing with those values separately

In [36]:
# df_5.trait.unique()

In [37]:
traits_to_keep = ['flowering_time', 'flag_leaf_emergence_time', 'light_intensity_PAR', 'aboveground_dry_biomass']

In [38]:
df_6 = df_5.loc[df_5.trait.isin(traits_to_keep)]
df_6.shape

(2338, 9)

In [39]:
df_6.isnull().sum()

date        0
sitename    0
range       0
column      0
lat         0
lon         0
cultivar    0
trait       0
value       0
dtype: int64

In [42]:
# df_6.head()

Double check that there are no E W subplots in this smaller dataset.

In [ ]:
e_w_plot_traits

Traits in `traits_to_keep` that also had E W subplot designations

In [ ]:
set(e_w_plot_traits) & set(traits_to_keep)

### Add TO Mappings

#### Final Steps
* Create `.csv`

In [43]:
need_to_create_csv = False

if need_to_create_csv:

    timestamp = datetime.datetime.now().replace(microsecond=0).isoformat()
    output_filename = f'tall_format_traits_{timestamp}.csv'.replace(':', '')
    df_6.to_csv(f'data/processed/{output_filename}')

## Not Using Currently

### Add Cumulative GDD
Note: this data is from the MAC Weather Station
* compare / use new geostreams data in weather data notebook, supplemented with Weather Station data

In [ ]:
# gdd_df = pd.read_csv('data/processed/daily_temps_gdd_2019-12-16T120237.csv')
# gdd_df.head()

In [ ]:
# Check for any days that would have a daily value of less than zero

# daily_value_list = ((gdd_df.air_temp_min + gdd_df.air_temp_max) / 2) - 10
# print(len(gdd_df))
# print(len(daily_value_list))

In [ ]:
# should have no output

# for i in daily_value_list:
    
#     if i < 0:
#         print(i)

In [ ]:
# date_list = gdd_df.date.values
# gdd_list = gdd_df.gdd.values

# date_gdd_dict = dict(zip(date_list, gdd_list))
# date_gdd_dict